In [4]:
pip install qgrid

In [5]:
#imports
import math
import scipy as sc
import numpy.matlib
import numpy as np
import operator
import pandas as pd
import qgrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from numpy import random
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import scale
#from sklearn.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import time
from sklearn.feature_selection import RFE



Juan3


In [6]:
from sklearn import datasets
db = np.loadtxt('./letter-recognition.data',delimiter=',',skiprows=0,dtype=bytes).astype(str)
X = db[0::, 1::] # extracting
Y = db[0::,0]
j=0;
for i in Y:
    Y[j]=ord(i) #chr(i)
    j+=1
data=X.astype(float)
Y=Y.astype(float)
X=X.astype(float)
print(Y)
print(X)

[84. 73. 68. ... 84. 83. 65.]
[[ 2.  8.  3. ...  8.  0.  8.]
 [ 5. 12.  3. ...  8.  4. 10.]
 [ 4. 11.  6. ...  7.  3.  9.]
 ...
 [ 6.  9.  6. ... 12.  2.  4.]
 [ 2.  3.  4. ...  9.  5.  8.]
 [ 4.  9.  6. ...  7.  2.  8.]]


In [7]:
from sklearn.naive_bayes import MultinomialNB
#Naive Bayes
#Validamos el modelo
Folds = 4
random.seed(19680801)
ETrain = np.zeros(Folds)
EVal = np.zeros(Folds)
Eficiencia=np.zeros(Folds)
Precision=np.zeros(Folds)
Sensibilidad=np.zeros(Folds)
Fscore=np.zeros(Folds)
skf = StratifiedKFold(n_splits=Folds)
j = 0
model = MultinomialNB()

for train, test in skf.split(X, Y):
    Xtrain = X[train,:]
    Ytrain = Y[train]
    Xtest = X[test,:]
    Ytest = Y[test]
    print(Ytest)
    #Normalizamos los datos
    media = np.mean(Xtrain)
    desvia = np.std(Xtrain)
    #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
    model.fit(Xtrain,Ytrain)  
    #Validación
    Ytrain_pred = model.predict(Xtrain)
    Yest = model.predict(Xtest)
    print(Yest)
    print(Yest.shape)
    #Evaluamos las predicciones del modelo con los datos de test
    
    Eficiencia[j] = accuracy_score(Ytest, Yest)
    Precision[j] =np.mean(precision_score(Ytest, Yest, average=None))
    Sensibilidad[j]=np.mean(recall_score(Ytest, Yest, average=None))
    Fscore[j]=np.mean(f1_score(Ytest, Yest, average=None))
    
    ETrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
    EVal[j] = np.mean(Yest.ravel() == Ytest.ravel())    
    j += 1
print('Eficiencia= '+str(Eficiencia)) 
print('Precisiòn= '+str(Precision)) 
print('Sensibilidad='+str(Sensibilidad)) 
print('Fscore= '+str(Fscore)) 
print('Eficiencia durante el entrenamiento = ' + str(np.mean(ETrain)) + '+-' + str(np.std(ETrain)))
print('Eficiencia durante la validación = ' + str(np.mean(EVal)) + '+-' + str(np.std(EVal)))

[84. 73. 68. ... 87. 87. 87.]
[73. 65. 65. ... 87. 87. 87.]
(5000,)
[74. 74. 72. ... 87. 87. 87.]
[74. 74. 88. ... 77. 87. 87.]
(5000,)
[72. 84. 72. ... 81. 82. 81.]
[75. 66. 72. ... 81. 67. 81.]
(5000,)
[66. 66. 86. ... 84. 83. 65.]
[66. 66. 80. ... 84. 90. 65.]
(5000,)
Eficiencia= [0.5566 0.554  0.5578 0.545 ]
Precisiòn= [0.56493613 0.55649172 0.56124294 0.54879155]
Sensibilidad=[0.55522429 0.55251394 0.55593474 0.54365455]
Fscore= [0.54482405 0.54191875 0.54597781 0.53180846]
Eficiencia durante el entrenamiento = 0.5536833333333333+-0.0020866640042581474
Eficiencia durante la validación = 0.55335+-0.005012733785071748


In [8]:
#Random Forest
def entrenamientoRF(arboles , variablesPorNodo, modelo):
    from sklearn.ensemble import RandomForestClassifier as RF
    from sklearn.ensemble import BaggingClassifier as BC

    #Validamos el modelo
    Folds = 4
    random.seed(19680801)
    ETrain = np.zeros(Folds)
    EVal = np.zeros(Folds)
    Eficiencia=np.zeros(Folds)
    Precision=np.zeros(Folds)
    Sensibilidad=np.zeros(Folds)
    Fscore=np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if modelo==0:
            #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
            model = RF(n_estimators=arboles,max_features=variablesPorNodo)  
            model.fit(Xtrain,Ytrain)
            #Validación
            Ytrain_pred = model.predict(Xtrain)
            Yest = model.predict(Xtest) 
        if modelo==1:
            model = BC(n_estimators=arboles,max_features=variablesPorNodo*16 )
            model.fit(Xtrain,Ytrain)
            #Validación
            Ytrain_pred = model.predict(Xtrain)
            Yest = model.predict(Xtest) 
        
        Eficiencia[j] = accuracy_score(Ytest, Yest)
        Precision[j] =np.mean(precision_score(Ytest, Yest, average=None))
        Sensibilidad[j]=np.mean(recall_score(Ytest, Yest, average=None))
        Fscore[j]=np.mean(f1_score(Ytest, Yest, average=None))
    
        #Evaluamos las predicciones del modelo con los datos de test
        ETrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        j += 1

    
    eficiencia = np.mean(EVal)
    intervaloConfianza = np.std(EVal)
    return eficiencia, intervaloConfianza, np.mean(Eficiencia),np.mean(Precision), np.mean(Sensibilidad), np.mean(Fscore)

In [9]:
#Calculando Numero de arboles por Variables analizadas por nodo
randn = np.random.randn
df_types2 = pd.DataFrame({
    'Numero de arboles' : pd.Series([5,5,5,5,5,5,10,10,10,10,10,10,20,20,20,20,20,20,50,50,50,50,50,50,100,100,100,100,100,100]), 'Variables analizadas por nodo' : pd.Series([3,6,9,12,15,16,3,6,9,12,15,16,3,6,9,12,15,16,3,6,9,12,15,16,3,6,9,12,15,16])})
df_types2["Eficiencia en validacion"] = ""
df_types2["Intervalo de confianza"] = ""
df_types2["Eficiencia"] = ""
df_types2["Precisiòn"] = ""
df_types2["Sensibilidad"] = ""
df_types2["Fscore"] = ""
#df_types.sort_index(inplace=True)
df_types2.set_index(['Numero de arboles','Variables analizadas por nodo'], inplace=True)

arrEficienciaVal = []
arrInConfianza= []
arrEficiencia=[]
arrPrecision=[]
arrSensibilidad=[]
arrFscore=[]
for item in df_types2.index:
    eficienciaVal, intervaloConfianza,Eficiencia, Precision, Sensibilidad, Fscore= entrenamientoRF(int(item[0]),int(item[1]),0)
    arrEficienciaVal.append(eficienciaVal)
    arrInConfianza.append(intervaloConfianza)
    arrEficiencia.append(Eficiencia)
    arrPrecision.append(Precision)
    arrSensibilidad.append(Sensibilidad)
    arrFscore.append(Fscore)
    
df_types2["Eficiencia en validacion"] = arrEficienciaVal
df_types2["Intervalo de confianza"] = arrInConfianza
df_types2["Eficiencia"] = arrEficiencia
df_types2["Precisiòn"] = arrPrecision
df_types2["Sensibilidad"] = arrSensibilidad
df_types2["Fscore"] = arrFscore
print("calula")

calula


In [10]:
qgrid_widget = qgrid.show_grid(df_types2, show_toolbar=False)
qgrid_widget
qgrid_widget.get_changed_df()

Eficiencia en validacion  ...    Fscore
Numero de arboles Variables analizadas por nodo                            ...          
5                 3                                               0.90130  ...  0.901748
                  6                                               0.90885  ...  0.909076
                  9                                               0.90870  ...  0.908918
                  12                                              0.90755  ...  0.907777
                  15                                              0.90535  ...  0.905580
                  16                                              0.90375  ...  0.904042
10                3                                               0.93360  ...  0.933565
                  6                                               0.93755  ...  0.937591
                  9                                               0.93450  ...  0.934480
                  12                                              0.93100  ...  0.931051
                  15                                              0.92685  ...  0.926765
                  16                                              0.92410  ...  0.924019
20                3                                               0.95120  ...  0.951069
                  6                                               0.94910  ...  0.948968
                  9                                               0.94790  ...  0.947815
                  12                                              0.93995  ...  0.939811
                  15                                              0.93870  ...  0.938629
                  16                                              0.93555  ...  0.935477
50                3                                               0.95985  ...  0.959757
                  6                                               0.95820  ...  0.958105
                  9                                               0.95415  ...  0.954014
                  12                                              0.94770  ...  0.947590
                  15                                              0.94220  ...  0.942050
                  16                                              0.94085  ...  0.940723
100               3                                               0.96330  ...  0.963188
                  6                                               0.96085  ...  0.960727
                  9                                               0.95615  ...  0.956009
                  12                                              0.95200  ...  0.951839
                  15                                              0.94590  ...  0.945762
                  16                                              0.94265  ...  0.942496

[30 rows x 6 columns]

In [11]:
#K vecinos 
from sklearn.neighbors import KNeighborsClassifier
def entrenamientoKnn(k):
    from sklearn.ensemble import RandomForestClassifier as RF
    from sklearn.ensemble import BaggingClassifier as BC

    #Validamos el modelo
    Folds = 4
    random.seed(19680801)
    ETrain = np.zeros(Folds)
    EVal = np.zeros(Folds)
    Eficiencia=np.zeros(Folds)
    Precision=np.zeros(Folds)
    Sensibilidad=np.zeros(Folds)
    Fscore=np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)
        knn = KNeighborsClassifier(n_neighbors=k)
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        knn.fit(Xtrain, Ytrain)
        Yest = knn.predict(Xtest)
        
        Eficiencia[j] = accuracy_score(Ytest, Yest)
        Precision[j] =np.mean(precision_score(Ytest, Yest, average=None))
        Sensibilidad[j]=np.mean(recall_score(Ytest, Yest, average=None))
        Fscore[j]=np.mean(f1_score(Ytest, Yest, average=None))
    
        #Evaluamos las predicciones del modelo con los datos de test
        EVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        j += 1

    
    eficiencia = np.mean(EVal)
    intervaloConfianza = np.std(EVal)
    return eficiencia, intervaloConfianza, np.mean(Eficiencia),np.mean(Precision), np.mean(Sensibilidad), np.mean(Fscore)

In [12]:
#Calculando #de vecinos
randn = np.random.randn
df_types2 = pd.DataFrame({
    'Numero de vecinos' : pd.Series(['1', '2', '3', '4', '5', '6', '7'])})

df_types2["Eficiencia en validacion"] = ""
df_types2["Intervalo de confianza"] = ""
df_types2["Eficiencia"] = ""
df_types2["Precisiòn"] = ""
df_types2["Sensibilidad"] = ""
df_types2["Fscore"] = ""

df_types2.set_index(['Numero de vecinos'], inplace=True)

arrEficienciaVal = []
arrInConfianza= []
arrEficiencia=[]
arrPrecision=[]
arrSensibilidad=[]
arrFscore=[]

qgrid_widget = qgrid.show_grid(df_types2, show_toolbar=False)
qgrid_widget

for h in df_types2.index:
    
    eficienciaVal, intervaloConfianza,Eficiencia, Precision, Sensibilidad, Fscore = entrenamientoKnn(int(h))
    
    arrEficienciaVal.append(eficienciaVal)
    arrInConfianza.append(intervaloConfianza)
    arrEficiencia.append(Eficiencia)
    arrPrecision.append(Precision)
    arrSensibilidad.append(Sensibilidad)
    arrFscore.append(Fscore)
        
df_types2["Eficiencia en validacion"] = arrEficienciaVal
df_types2["Intervalo de confianza"] = arrInConfianza
df_types2["Eficiencia"] = arrEficiencia
df_types2["Precisiòn"] = arrPrecision
df_types2["Sensibilidad"] = arrSensibilidad
df_types2["Fscore"] = arrFscore

In [13]:
qgrid_widget = qgrid.show_grid(df_types2, show_toolbar=False)
qgrid_widget
qgrid_widget.get_changed_df()

,Eficiencia en validacion,Intervalo de confianza,Eficiencia,Precisiòn,Sensibilidad,Fscore
Numero de vecinos,,,,,,
1,0.95460,0.003855,0.95460,0.954671,0.954400,0.954407
2,0.94225,0.004337,0.94225,0.944808,0.942053,0.942398
3,0.95190,0.003015,0.95190,0.952715,0.951709,0.951847
4,0.94970,0.002476,0.94970,0.950740,0.949502,0.949657
5,0.94980,0.002657,0.94980,0.950575,0.949529,0.949703
6,0.94720,0.001225,0.94720,0.948268,0.946939,0.947140
7,0.94670,0.000539,0.94670,0.947547,0.946397,0.946592


In [14]:
#Redes Neuronales Artificiales
def entrenamientoRNA(nroCapasOcultas, neuronasPorCapa):
    from sklearn.neural_network import MLPClassifier
    from sklearn.model_selection import StratifiedKFold

    Folds = 4
    random.seed(19680801)
    ETrain = np.zeros(Folds)
    EVal = np.zeros(Folds)
    Eficiencia=np.zeros(Folds)
    Precision=np.zeros(Folds)
    Sensibilidad=np.zeros(Folds)
    Fscore=np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento

        if(nroCapasOcultas==1):
            mlp=MLPClassifier(activation='tanh',max_iter = 700,hidden_layer_sizes=(neuronasPorCapa))
        else:
            mlp=MLPClassifier(activation='tanh',max_iter = 700,hidden_layer_sizes=(neuronasPorCapa,neuronasPorCapa))
        
        mlp.fit(Xtrain,Ytrain)
        
        Ytrain_pred = mlp.predict(Xtrain)
        Yest = mlp.predict(Xtest)          
        
        Eficiencia[j] = accuracy_score(Ytest, Yest)
        Precision[j] =np.mean(precision_score(Ytest, Yest, average=None))
        Sensibilidad[j]=np.mean(recall_score(Ytest, Yest, average=None))
        Fscore[j]=np.mean(f1_score(Ytest, Yest, average=None))
    
        #Evaluamos las predicciones del modelo con los datos de test
        EVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        j += 1

    
    eficiencia = np.mean(EVal)
    intervaloConfianza = np.std(EVal)
    return eficiencia, intervaloConfianza, np.mean(Eficiencia),np.mean(Precision), np.mean(Sensibilidad), np.mean(Fscore)

In [15]:
#Aplicando Redes Neuronales Artificiales 
df_types2 = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,2,2,2,2,2]),
    'Neuronas por capa' : pd.Series([20,24,28,32,36,20,24,28,32,36])})
df_types2["Eficiencia en validacion"] = ""
df_types2["Intervalo de confianza"] = ""
df_types2["Eficiencia"] = ""
df_types2["Precisiòn"] = ""
df_types2["Sensibilidad"] = ""
df_types2["Fscore"] = ""

df_types2.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)

arrEficienciaVal = []
arrInConfianza= []
arrEficiencia=[]
arrPrecision=[]
arrSensibilidad=[]
arrFscore=[]

for h in df_types2.index:
    eficienciaVal, intervaloConfianza,Eficiencia, Precision, Sensibilidad, Fscore = entrenamientoRNA(int(h[0]),int(h[1]))   
    arrEficienciaVal.append(eficienciaVal)
    arrInConfianza.append(intervaloConfianza)
    arrEficiencia.append(Eficiencia)
    arrPrecision.append(Precision)
    arrSensibilidad.append(Sensibilidad)
    arrFscore.append(Fscore)
        
df_types2["Eficiencia en validacion"] = arrEficienciaVal
df_types2["Intervalo de confianza"] = arrInConfianza
df_types2["Eficiencia"] = arrEficiencia
df_types2["Precisiòn"] = arrPrecision
df_types2["Sensibilidad"] = arrSensibilidad
df_types2["Fscore"] = arrFscore

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (700) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [16]:
qgrid_widget = qgrid.show_grid(df_types2, show_toolbar=False)
qgrid_widget
qgrid_widget.get_changed_df()

Eficiencia en validacion  ...    Fscore
N. de capas ocultas Neuronas por capa                            ...          
1                   20                                  0.84175  ...  0.841212
                    24                                  0.86885  ...  0.868460
                    28                                  0.87990  ...  0.879529
                    32                                  0.88705  ...  0.886565
                    36                                  0.89765  ...  0.897244
2                   20                                  0.87815  ...  0.877634
                    24                                  0.90140  ...  0.901226
                    28                                  0.91435  ...  0.914240
                    32                                  0.91855  ...  0.918272
                    36                                  0.92875  ...  0.928571

[10 rows x 6 columns]

In [17]:
#Maquina de vectores
from sklearn import preprocessing
from sklearn.svm import SVC
def entrenamientoMV(kernel, c, gamma):

    Folds = 4
    random.seed(19680801)
    ETrain = np.zeros(Folds)
    EVal = np.zeros(Folds)
    Eficiencia=np.zeros(Folds)
    Precision=np.zeros(Folds)
    Sensibilidad=np.zeros(Folds)
    Fscore=np.zeros(Folds)
    Porcentaje=np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):   
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento        
        if( kernel == 'linear'):
            modelo = SVC(kernel= kernel, C=c, decision_function_shape='ovo')
        else:
            modelo = SVC(kernel= kernel, C=c, gamma=gamma, decision_function_shape='ovo') 
      
        modelo.fit(Xtrain,Ytrain)
        
        Ytrain_pred = modelo.predict(Xtrain)       
        Yest = modelo.predict(Xtest)          
        
        Eficiencia[j] = accuracy_score(Ytest, Yest)
        Precision[j] =np.mean(precision_score(Ytest, Yest, average=None))
        Sensibilidad[j]=np.mean(recall_score(Ytest, Yest, average=None))
        Fscore[j]=np.mean(f1_score(Ytest, Yest, average=None))
    
        #Evaluamos las predicciones del modelo con los datos de test
        EVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        Porcentaje[j] =  len(modelo.support_vectors_) / len(Xtrain)
        j += 1

    
    eficiencia = np.mean(EVal)
    intervaloConfianza = np.std(EVal)
    porcentaje = np.mean(Porcentaje)
    
    return eficiencia, intervaloConfianza, np.mean(Eficiencia),np.mean(Precision), np.mean(Sensibilidad), np.mean(Fscore), porcentaje

In [18]:
#Aplicando maquinas de Vectores
df_types2 = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types2["% de Vectores de Soporte"] = ""
df_types2["Eficiencia en validacion"] = ""
df_types2["Intervalo de confianza"] = ""
df_types2["Eficiencia"] = ""
df_types2["Precisiòn"] = ""
df_types2["Sensibilidad"] = ""
df_types2["Fscore"] = ""

df_types2.set_index(['Kernel','C','gamma'], inplace=True)
#df_types.sort_index(inplace=True)

arrEficienciaVal = []
arrInConfianza= []
arrEficiencia=[]
arrPrecision=[]
arrSensibilidad=[]
arrFscore=[]
arrPorcentaje=[]

for h in df_types2.index:
    eficienciaVal, intervaloConfianza,Eficiencia, Precision, Sensibilidad, Fscore, porcentaje = entrenamientoMV(h[0],h[1],h[2])   
    arrEficienciaVal.append(eficienciaVal)
    arrInConfianza.append(intervaloConfianza)
    arrEficiencia.append(Eficiencia)
    arrPrecision.append(Precision)
    arrSensibilidad.append(Sensibilidad)
    arrFscore.append(Fscore)
    arrPorcentaje.append(porcentaje)
        
df_types2["Eficiencia en validacion"] = arrEficienciaVal
df_types2["Intervalo de confianza"] = arrInConfianza
df_types2["Eficiencia"] = arrEficiencia
df_types2["Precisiòn"] = arrPrecision
df_types2["Sensibilidad"] = arrSensibilidad
df_types2["Fscore"] = arrFscore
df_types2["% de Vectores de Soporte"] = arrPorcentaje

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Pr

In [19]:
qgrid_widget = qgrid.show_grid(df_types2, show_toolbar=False)
qgrid_widget
qgrid_widget.get_changed_df()

% de Vectores de Soporte  ...    Fscore
Kernel C       gamma                            ...          
linear 0.001   0.00                   0.867550  ...  0.770046
       0.010   0.00                   0.636733  ...  0.831312
       0.100   0.00                   0.474150  ...  0.848782
       1.000   0.00                   0.411817  ...  0.852746
       10.000  0.00                   0.396083  ...  0.849128
       100.000 0.00                   0.394217  ...  0.849053
rbf    0.001   0.01                   1.000000  ...  0.003005
               0.10                   0.999650  ...  0.003005
               1.00                   0.999600  ...  0.003005
       0.010   0.01                   0.996317  ...  0.508149
               0.10                   0.999700  ...  0.003005
               1.00                   0.999817  ...  0.003005
       0.100   0.01                   0.828183  ...  0.830865
               0.10                   0.871383  ...  0.881012
               1.00                   0.999650  ...  0.011781
       1.000   0.01                   0.557583  ...  0.937430
               0.10                   0.647033  ...  0.970970
               1.00                   0.963450  ...  0.587504
       10.000  0.01                   0.409483  ...  0.969027
               0.10                   0.643633  ...  0.974254
               1.00                   0.944717  ...  0.624124
       100.000 0.01                   0.389833  ...  0.970169
               0.10                   0.643650  ...  0.974355
               1.00                   0.944717  ...  0.624124

[24 rows x 7 columns]

In [20]:
def entrenamiento_sin_seleccion_caracteristicas(splits, X, Y):
   
    #Implemetamos la metodología de validación
    Errores = np.ones(splits)
    Score = np.ones(splits)
    times = np.ones(splits)
    j = 0
    kf = KFold(n_splits=splits)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        #Creamos el clasificador SVM.
        clf = SVC(kernel="linear", C=1)
        #Aquí se entran y se valida el modelo sin hacer selección de características
        ######
        # Entrenamiento el modelo.
        #Para calcular el costo computacional
        tiempo_i = time.time()
        clf.fit(X_train,y_train)
        # Validación del modelo
        Errores[j] = accuracy_score(y_true=y_test, y_pred=clf.predict(X_test))
        times[j] = time.time()-tiempo_i
        j+=1

    return clf, np.mean(Errores), np.std(Errores), np.mean(times)

In [21]:
def recursive_feature_elimination_wrapper(estimator, feature_numbers, X,Y):
    rfe = RFE(estimator=estimator, step=1,n_features_to_select=feature_numbers)
    tiempo_i = time.time()
    rfe.fit(X=X,y=Y)
    time_o = time.time()-tiempo_i
    feature_mask = rfe.support_
    #print(feature_mask)
    features_rank = rfe.ranking_
    #print(features_rank)
    estimator = rfe.estimator_
    #print(estimator)
    #print(rfe)
    
    return rfe, feature_mask, features_rank, estimator, time_o

In [22]:
def experimentar(n_feats, n_sets, X, Y):
    df = pd.DataFrame()
    idx = 0
    for split_number in n_sets: 
    #Sin selección de características
        # se ignorar las otras salidas
        _,err,ic,t_ex = entrenamiento_sin_seleccion_caracteristicas(split_number, X,Y)  
        df.loc[idx,'CON_SEL'] = 'NO'
        df.loc[idx,'NUM_VAR'] = X.shape[1]
        df.loc[idx,'NUM_SPLITS'] = split_number
        df.loc[idx,'ERROR_VALIDACION'] = err
        df.loc[idx,'IC_STD_VALIDACION'] = ic
        df.loc[idx,'T_EJECUCION'] = t_ex
        idx+=1
    print("termina experimentos sin selección")
    #Con selección de características
    for f in n_feats:
        for split_number in n_sets:
            #Implemetamos la metodología de validación 
            Errores = np.ones(split_number)
            Score = np.ones(split_number)
            times = np.ones(split_number)
            kf = KFold(n_splits=split_number)
            j = 0
            for train_index, test_index in kf.split(X):
                
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = Y[train_index], Y[test_index]
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)
                
                svc =  SVC(kernel="linear", C=1)
                
                # se ignorar las otras salidas
                rfe, _, _, _, t = recursive_feature_elimination_wrapper(estimator=svc,feature_numbers=f, X=X_test,Y=y_test)
            
                Errores[j]=accuracy_score(y_true = y_test,  y_pred=rfe.predict(X_test))
                times[j] = t
                j+=1

            df.loc[idx,'CON_SEL'] = 'SI'
            df.loc[idx,'NUM_VAR'] = f
            df.loc[idx,'NUM_SPLITS'] = split_number
            df.loc[idx,'ERROR_VALIDACION'] = np.mean(Errores)
            df.loc[idx, 'IC_STD_VALIDACION'] = np.std(Errores)
            df.loc[idx, 'T_EJECUCION'] = np.mean(times)
            idx+=1
    return df

In [23]:
dfr = experimentar(n_feats = [3, 5,7, 10,15,17], n_sets = [5, 10], X= X, Y=Y)

termina experimentos sin selección


In [24]:
dfr.sort_values(['ERROR_VALIDACION','T_EJECUCION'], ascending=[False, True])

,CON_SEL,NUM_VAR,NUM_SPLITS,ERROR_VALIDACION,IC_STD_VALIDACION,T_EJECUCION
13,SI,17.0,10.0,0.89775,0.010048,0.125285
11,SI,15.0,10.0,0.89200,0.011129,0.250520
12,SI,17.0,5.0,0.88285,0.007372,0.348261
10,SI,15.0,5.0,0.87805,0.006794,0.683361
1,NO,16.0,10.0,0.85265,0.010191,4.573206
0,NO,16.0,5.0,0.85175,0.006448,4.702637
9,SI,10.0,10.0,0.81500,0.020971,0.790099
8,SI,10.0,5.0,0.79035,0.011048,2.244891
7,SI,7.0,10.0,0.73375,0.015592,1.060555
6,SI,7.0,5.0,0.72935,0.012492,2.988781


In [25]:
svc =  SVC(kernel="linear", C=1)
rfe, feature_mask, _, _, _ = recursive_feature_elimination_wrapper(svc,17, X,Y)
print("esta es la mascara (deberia ser solo valores True y False) \n", feature_mask)

esta es la mascara (deberia ser solo valores True y False) 
 [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


In [32]:
#Indice de fisher
from sklearn.feature_selection import SelectKBest, f_classif
valores = f_classif( X , Y )
print(valores)

X_new = SelectKBest(f_classif, k=1).fit(X, Y)
caracteristicas=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
atrib = X_new.get_support()
params=X_new.get_params()
atributos = [caracteristicas[i] for i in list(atrib.nonzero()[0])]
print(atrib)
atributos

(array([  76.01867455,    3.33815995,  129.61413459,   10.33693443,
        110.85593313,  349.0919552 , 1172.18344478,  407.21061923,
        752.44065518,  349.47724735, 1241.35211317,  915.64649155,
       1080.23983158,  917.06275919,  640.67016809,  283.45775286]), array([0.00000000e+00, 3.40266361e-08, 0.00000000e+00, 2.32693754e-40,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))
[False False False False False False False False False False  True False
 False False False False]


[11]

In [26]:
def MAPE(Y_est,Y):
    """Mean Absolute Percentage Error para los problemas de regresión
    Y_est: numpy array con los valores estimados
    Y: numpy array con las etiquetas verdaderas
    retorna: mape
    """
    N = np.size(Y)
    mape = np.sum(abs((Y_est.reshape(N,1) - Y.reshape(N,1))/Y.reshape(N,1)))/N
    return mape 

In [27]:
def kernel_gaussiano(x):
    return (np.exp((-0.5)*x**2))

def ParzenWindow(x,Data,h,Datay=None):
    """"ventana de parzen
    x: vector con representando una sola muestra
    Data: vector de muestras de entrenamiento
    h: ancho de la ventana de kernel
    Datay: vector con los valores de salida (y), Si no se pasa como argumento, 
        se calcula un ventana de parzen sin multiplicar los valores de este vector.
    retorna: el valor de ventana de parzen para una muestra
    """
    h = h
    Ns = Data.shape[0]
    suma = 0
    for k in range(Ns):
        u = euclidean(x,Data[k,:])
        if Datay is None:
            suma += kernel_gaussiano(u/h)
        else:
            suma += kernel_gaussiano(u/h)*Datay[k]
    return suma

In [28]:
def Nadaraya_Watson(X_train, Y_train, X_test, h):
    import scipy
    """ Funcion que implementa metodo de ventana de parzen para
        para clasificación
    X_train: es la matriz con las muestras de entrenamiento
    Y_train: es un vector con los valores de salida pra cada una de las muestras de entrenamiento
    X_test: es la matriz con las muestras de validación
    h (float): ancho de h de la ventana
    retorna: - las estimaciones del modelo parzen para el conjunto X_test 
              esta matriz debe tener un shape de [row/muestras de X_test]
             - las probabilidades de la vetana [row/muestras de X_test, numero de clases]  
    """
    
    Yest = np.zeros(X_test.shape[0])
    distancias = scipy.spatial.distance.cdist(X_test, X_train) / h
    #parzen=ParzenWindow(X_test,X_train,h,Y_train)
    for i in range(len(X_test)):
        pesos = kernel_gaussiano(distancias[i])
        x = np.sum(np.multiply(pesos, Y_train))
        y = np.sum(pesos)
        Yest[i] = x/y   
    
    #Debe retornar un vector que contenga las predicciones para cada una de las muestras en X_val, en el mismo orden.  
    return Yest

In [30]:
def experimentarParzen (X, Y, hs):
    """Función que realiza los experimentos con knn usando
       una estrategia de validacion entrenamiento y pruebas
    X: matriz de numpy conjunto con muestras y caracteristicas
    Y: vector de numpy con los valores de las etiquetas
    ks: List[int/float] lista con los valores de k-vecinos a usar
    retorna: dataframe con los resultados, debe contener las siguientes columnas:
        - el ancho de ventana, 
        - el error medio de prueba
        - la desviacion estandar del error
        - número de promedio en el conjunto de prueba/validacion
    """
    # se usa la función para implementar la estrategia de validación.
    kfolds = KFold(n_splits=26)
    resultados = pd.DataFrame()
    idx = 0
    # iteramos sobre los valores de hs
    for h in hs:
        # lista para almacenar los errores y numero de muestras
        # de cada iteración
        # de la validación
        error_temp = []
        numero_muestras = []
        
        for train, test in kfolds.split(X,Y ):

            Xtrain = X[train,:]
            Ytrain = Y[train]
            Xtest = X[test,:]
            Ytest = Y[test]
            #normalizamos los datos
            scaler = StandardScaler()
            scaler.fit(Xtrain)
            Xtrain = scaler.transform(Xtrain)
            Xtest = scaler.transform(Xtest)
            Yest = Nadaraya_Watson(Xtrain,Ytrain,Xtest,h)
            errorTest = MAPE(Yest,Ytest)
            error_temp.append(errorTest)
            numero_muestras.append(len(Xtest))
            
    
        resultados.loc[idx,'ancho de ventana'] = h 
        resultados.loc[idx,'error de prueba(media)'] = np.mean(error_temp)
        resultados.loc[idx,'error de prueba(desviación estandar)'] =np.std(error_temp)
        resultados.loc[idx,'muestras en conjunto de pruebas (media)'] = np.mean(numero_muestras)
       
        idx+=1
    return (resultados)

In [31]:
hs = [1,1.5 ,2.5, 5, 10]
experimentos_parzen = experimentarParzen(X,Y, hs)
experimentos_parzen

,ancho de ventana,error de prueba(media),error de prueba(desviación estandar),muestras en conjunto de pruebas (media)
0,1.0,0.043353,0.001262,769.230769
1,1.5,0.064175,0.001599,769.230769
2,2.5,0.078128,0.001947,769.230769
3,5.0,0.083326,0.002047,769.230769
4,10.0,0.084664,0.002066,769.230769
